In [63]:
%%writefile sort.cpp

#include <iostream>
#include <cstdlib>
#include <omp.h>

using namespace std;

// ====================== Bubble Sort ==========================

// Sequential Bubble Sort
void sequentialBubbleSort(int arr[], int n) {
    for (int i = 0; i < n - 1; ++i)
        for (int j = 0; j < n - i - 1; ++j)
            if (arr[j] > arr[j + 1])
                swap(arr[j], arr[j + 1]);
}

// Parallel Bubble Sort using Odd-Even Transposition
void parallelBubbleSort(int arr[], int n) {
    for (int i = 0; i < n; ++i) {
        int phase = i % 2;
        #pragma omp parallel for
        for (int j = phase; j < n - 1; j += 2) {
            // int tid = omp_get_thread_num();
            // cout << "Thread " << tid << " working on index " << j << endl;
            if (arr[j] > arr[j + 1])
                swap(arr[j], arr[j + 1]);
        }
    }
}

// ===================== Merge Sort Helpers ======================

// Merge utility for both versions
void merge(int arr[], int l, int m, int r) {
    int n1 = m - l + 1;
    int n2 = r - m;

    int *L = new int[n1];
    int *R = new int[n2];

    for (int i = 0; i < n1; ++i)
        L[i] = arr[l + i];
    for (int j = 0; j < n2; ++j)
        R[j] = arr[m + 1 + j];

    int i = 0, j = 0, k = l;
    while (i < n1 && j < n2)
        arr[k++] = (L[i] <= R[j]) ? L[i++] : R[j++];
    while (i < n1)
        arr[k++] = L[i++];
    while (j < n2)
        arr[k++] = R[j++];

    delete[] L;
    delete[] R;
}

// ====================== Merge Sort ============================

// Sequential Merge Sort
void sequentialMergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;
        sequentialMergeSort(arr, l, m);
        sequentialMergeSort(arr, m + 1, r);
        merge(arr, l, m, r);
    }
}

// // Parallel Merge Sort
// void parallelMergeSort(int arr[], int l, int r) {
//     if (l < r) {
//         int m = l + (r - l) / 2;

//         #pragma omp parallel sections
//         {
//             #pragma omp section
//             {
//                 // int tid = omp_get_thread_num();
//                 // cout << "Thread " << tid << " sorting left half\n";
//                 parallelMergeSort(arr, l, m);
//             }

//             #pragma omp section
//             {
//                 // int tid = omp_get_thread_num();
//                 // cout << "Thread " << tid << " sorting right half\n";
//                 parallelMergeSort(arr, m + 1, r);
//             }
//         }

//         merge(arr, l, m, r);
//     }
// }

void parallelMergeSort(int arr[], int l, int r, int depth = 0) {
    if (l < r) {
        int m = l + (r - l) / 2;

        if (depth <= 3) {  // Limit depth to avoid thread explosion
            #pragma omp parallel sections
            {
                #pragma omp section
                parallelMergeSort(arr, l, m, depth + 1);
                #pragma omp section
                parallelMergeSort(arr, m + 1, r, depth + 1);
            }
        } else {
            sequentialMergeSort(arr, l, m);
            sequentialMergeSort(arr, m + 1, r);
        }

        merge(arr, l, m, r);
    }
}


// print the array
void printArray(int arr[], int n, const string& label) {
    cout << label << ": ";
    for (int i = 0; i < n; ++i)
        cout << arr[i] << " ";
    cout << endl;
}


// ====================== Main Driver ===========================

int main() {
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

    // Original data
    int *original = new int[n];
    for (int i = 0; i < n; ++i)
        original[i] = rand() % 1000;

    // Copies
    int *arrBubbleSeq = new int[n];
    int *arrBubblePar = new int[n];
    int *arrMergeSeq  = new int[n];
    int *arrMergePar  = new int[n];

    for (int i = 0; i < n; ++i)
        arrBubbleSeq[i] = arrBubblePar[i] = arrMergeSeq[i] = arrMergePar[i] = original[i];

    // printArray(original, n, "Original Array");
    cout<<endl;

    double start, end;

    int numThreads;
    cout << "Enter number of threads to use: ";
    cin >> numThreads;
    omp_set_num_threads(numThreads);
    cout << "Using " << numThreads << " threads for parallel operations.\n";


    // Sequential Bubble Sort
    start = omp_get_wtime();
    sequentialBubbleSort(arrBubbleSeq, n);
    end = omp_get_wtime();
    double time_seq_bubble = end - start;
    // printArray(arrBubbleSeq, n, "Sorted (Sequential Bubble)");

    // Parallel Bubble Sort
    start = omp_get_wtime();
    parallelBubbleSort(arrBubblePar, n);
    end = omp_get_wtime();
    double time_par_bubble = end - start;
    // printArray(arrBubblePar, n, "Sorted (Parallel Bubble)");

    // Sequential Merge Sort
    start = omp_get_wtime();
    sequentialMergeSort(arrMergeSeq, 0, n - 1);
    end = omp_get_wtime();
    double time_seq_merge = end - start;
    // printArray(arrMergeSeq, n, "Sorted (Sequential Merge)");

    // Parallel Merge Sort
    start = omp_get_wtime();
    parallelMergeSort(arrMergePar, 0, n - 1);
    end = omp_get_wtime();
    double time_par_merge = end - start;
    // printArray(arrMergePar, n, "Sorted (Parallel Merge)");

    // Results
    cout << "\nPerformance Comparison (in seconds):\n";
    cout << "Sequential Bubble Sort : " << time_seq_bubble << endl;
    cout << "Parallel Bubble Sort   : " << time_par_bubble << endl;
    cout << "Sequential Merge Sort  : " << time_seq_merge << endl;
    cout << "Parallel Merge Sort    : " << time_par_merge << endl;

    delete[] original;
    delete[] arrBubbleSeq;
    delete[] arrBubblePar;
    delete[] arrMergeSeq;
    delete[] arrMergePar;

    return 0;
}

// g++ -fopenmp sort.cpp -o sort
// ./sort

Overwriting sort.cpp


In [64]:
!g++ -fopenmp sort.cpp -o sort

In [65]:
!./sort

Enter the size of the array: 1000

Enter number of threads to use: 8
Using 8 threads for parallel operations.

Performance Comparison (in seconds):
Sequential Bubble Sort : 0.00689526
Parallel Bubble Sort   : 0.122757
Sequential Merge Sort  : 0.000318415
Parallel Merge Sort    : 0.000549075


In [46]:
%%writefile search.cpp

#include <iostream>
#include <vector>
#include <queue>
#include <stack>
#include <omp.h>
using namespace std;

vector<vector<int>> graph;

void addEdge(int src, int dest) {
    graph[src].push_back(dest);
    graph[dest].push_back(src);
}

void displayGraph(int numVertices) {
    cout << "\nGraph Representation:\n";
    for (int i = 0; i < numVertices; ++i) {
        cout << "Vertex " << i << ": ";
        for (int neighbor : graph[i]) {
            cout << neighbor << " ";
        }
        cout << endl;
    }
}

void printVector(const vector<int>& v) {
    for (int val : v) cout << val << " ";
    cout << endl <<endl <<endl;
}

void sequentialBFS(int start, int numVertices) {
    double t1 = omp_get_wtime();
    vector<bool> visited(numVertices, false);
    vector<int> result;
    queue<int> q;
    visited[start] = true;
    q.push(start);

    while (!q.empty()) {
        int current = q.front(); q.pop();
        result.push_back(current);
        for (int neighbor : graph[current]) {
            if (!visited[neighbor]) {
                visited[neighbor] = true;
                q.push(neighbor);
            }
        }
    }

    double t2 = omp_get_wtime();
    cout << "\nSequential BFS Output:\n";
    printVector(result);
    cout << "Time: " << (t2 - t1) << " seconds\n";
}

void parallelBFS(int start, int numVertices) {
    double t1 = omp_get_wtime();
    vector<bool> visited(numVertices, false);
    queue<int> q;
    visited[start] = true;
    q.push(start);
    vector<int> result;

    while (!q.empty()) {
        int current = q.front(); q.pop();

        #pragma omp critical
        result.push_back(current);

        vector<int> neighbors = graph[current];

        #pragma omp parallel for
        for (int i = 0; i < neighbors.size(); ++i) {
            int neighbor = neighbors[i];
            #pragma omp critical
            {
                if (!visited[neighbor]) {
                    visited[neighbor] = true;
                    q.push(neighbor);
                    cout << "Thread " << omp_get_thread_num() << " visiting node " << neighbor << endl;
                }
            }
        }
    }

    double t2 = omp_get_wtime();
    cout << "\nParallel BFS Output:\n";
    printVector(result);
    cout << "Time: " << (t2 - t1) << " seconds\n";
}

void sequentialDFS(int start, int numVertices) {
    double t1 = omp_get_wtime();
    vector<bool> visited(numVertices, false);
    stack<int> s;
    vector<int> result;
    visited[start] = true;
    s.push(start);

    while (!s.empty()) {
        int current = s.top(); s.pop();
        result.push_back(current);
        for (int neighbor : graph[current]) {
            if (!visited[neighbor]) {
                visited[neighbor] = true;
                s.push(neighbor);
            }
        }
    }

    double t2 = omp_get_wtime();
    cout << "\nSequential DFS Output:\n";
    printVector(result);
    cout << "Time: " << (t2 - t1) << " seconds\n";
}

void parallelDFS(int start, int numVertices) {
    double t1 = omp_get_wtime();
    vector<bool> visited(numVertices, false);
    stack<int> s;
    vector<int> result;
    visited[start] = true;
    s.push(start);

    while (!s.empty()) {
        int current = s.top(); s.pop();

        #pragma omp critical
        result.push_back(current);

        vector<int> neighbors = graph[current];

        #pragma omp parallel for
        for (int i = 0; i < neighbors.size(); ++i) {
            int neighbor = neighbors[i];
            #pragma omp critical
            {
                if (!visited[neighbor]) {
                    visited[neighbor] = true;
                    s.push(neighbor);
                    cout << "Thread " << omp_get_thread_num() << " visiting node " << neighbor << endl;
                }
            }
        }
    }

    double t2 = omp_get_wtime();
    cout << "\nParallel DFS Output:\n";
    printVector(result);
    cout << "Time: " << (t2 - t1) << " seconds\n";
}

int main() {
    int numVertices, numEdges;
    cout << "Enter number of vertices: ";
    cin >> numVertices;
    graph.resize(numVertices);

    cout << "Enter number of edges: ";
    cin >> numEdges;
    cout << "Enter edges (source destination):\n";
    for (int i = 0; i < numEdges; ++i) {
        int u, v;
        cin >> u >> v;
        addEdge(u, v);
    }

    displayGraph(numVertices);

    int startVertex;
    cout << "Enter start vertex: ";
    cin >> startVertex;

    int numThreads;
    cout << "Enter number of threads to use: ";
    cin >> numThreads;
    omp_set_num_threads(numThreads);
    cout << "Using " << numThreads << " threads for parallel operations.\n";

    // cout << "Default max threads: " << omp_get_max_threads() << endl;
    cout << "\n --Sequential BFS: \n";
    sequentialBFS(startVertex, numVertices);
    cout << "\n --Parallel BFS: \n";
    parallelBFS(startVertex, numVertices);
    cout << "\n --Sequential DFS: \n";
    sequentialDFS(startVertex, numVertices);
    cout << "\n --Parallel DFS: \n";
    parallelDFS(startVertex, numVertices);

    return 0;
}

Overwriting search.cpp


In [47]:
!g++ -fopenmp search.cpp -o search

In [45]:
!./search

Enter number of vertices: 7
Enter number of edges: 7
Enter edges (source destination):
0
1
0
2
0
5
1
3
1
4
5
6
2
5

Graph Representation:
Vertex 0: 1 2 5 
Vertex 1: 0 3 4 
Vertex 2: 0 5 
Vertex 3: 1 
Vertex 4: 1 
Vertex 5: 0 6 2 
Vertex 6: 5 
Enter start vertex: 0
Enter number of threads to use: 5
Using 5 threads for parallel operations.
Sequenrtial BFS: 
Sequential BFS Output:
0 1 2 5 3 4 6 


Time: 1.4028e-05 seconds
Parallel BFS: Thread 2 visiting node 5
Thread 1 visiting node 2
Thread 0 visiting node 1
Thread 1 visiting node 6
Thread 1 visiting node 3
Thread 2 visiting node 4

Parallel BFS Output:
0 5 2 1 6 3 4 


Time: 0.000586746 seconds
Sequenrtial DFS: 
Sequential DFS Output:
0 5 6 2 1 4 3 


Time: 1.2643e-05 seconds
Parallel DFS: Thread 0 visiting node 1
Thread 1 visiting node 2
Thread 2 visiting node 5
Thread 1 visiting node 6
Thread 1 visiting node 3
Thread 2 visiting node 4

Parallel DFS Output:
0 5 6 2 1 4 3 


Time: 0.000331413 seconds


In [73]:
%%writefile red.cpp

#include <iostream>
#include <omp.h>
#include <ctime>
#include <cstdlib>
#include <climits>
using namespace std;

// Find the minimum value
void find_min(int *arr, int n) {
    int min_val = INT_MAX;

    double start_time = omp_get_wtime(); // Start time
    #pragma omp parallel for reduction(min:min_val)
    for (int i = 0; i < n; i++) {
        #pragma omp critical
        {
            cout << "Thread = " << omp_get_thread_num() << " and i = " << i << endl;
        }
        if (arr[i] < min_val) {
            min_val = arr[i];
        }
    }
    double end_time = omp_get_wtime(); // End time
    cout << "\nMinimum value = " << min_val << endl;
    cout << "Time for finding minimum: " << end_time - start_time << " seconds" << endl;
}

// Find the maximum value
void find_max(int *arr, int n) {
    int max_val = INT_MIN;

    double start_time = omp_get_wtime(); // Start time
    #pragma omp parallel for reduction(max:max_val)
    for (int i = 0; i < n; i++) {
        #pragma omp critical
        {
            cout << "Thread = " << omp_get_thread_num() << " and i = " << i << endl;
        }
        if (arr[i] > max_val) {
            max_val = arr[i];
        }
    }
    double end_time = omp_get_wtime(); // End time
    cout << "\nMaximum value = " << max_val << endl;
    cout << "Time for finding maximum: " << end_time - start_time << " seconds" << endl;
}

// Compute the sum of all elements
void compute_sum(int *arr, int n) {
    long long sum = 0;

    double start_time = omp_get_wtime(); // Start time
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < n; i++) {
        sum += arr[i];
        #pragma omp critical
        {
            cout << "Thread " << omp_get_thread_num() << " and i = " << i << endl;
        }
    }
    double end_time = omp_get_wtime(); // End time
    cout << "\nSum = " << sum << endl;
    cout << "Time for computing sum: " << end_time - start_time << " seconds" << endl;
}

// Compute the average of the array
void compute_average(int *arr, int n) {
    long long sum = 0;

    double start_time = omp_get_wtime(); // Start time
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < n; i++) {
        sum += arr[i];
        #pragma omp critical
        {
            cout << "Thread " << omp_get_thread_num() << " and i = " << i << endl;
        }
    }
    double end_time = omp_get_wtime(); // End time
    double avg = static_cast<double>(sum) / n;
    cout << "\nAverage = " << avg << endl;
    cout << "Time for computing average: " << end_time - start_time << " seconds" << endl;
}

int main() {
    omp_set_num_threads(4);
    srand(time(0)); // Seed once

    int n;
    cout << "Enter the number of elements in the array: ";
    cin >> n;

    // User decides the number of threads
    int numThreads;
    cout << "Enter number of threads to use: ";
    cin >> numThreads;

    omp_set_num_threads(numThreads); // Set number of threads for OpenMP

    srand(time(0)); // Seed once


    int* arr = new int[n]; // Allocate array dynamically

    for (int i = 0; i < n; ++i) {
        arr[i] = rand() % 100; // Random values between 0 and 99
    }

    cout << "\nArray elements: ";
    for (int i = 0; i < n; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;

    // Perform operations with user-specified threads and measure time
    find_min(arr, n);
    find_max(arr, n);
    compute_sum(arr, n);    // Calculate sum separately
    compute_average(arr, n); // Calculate average separately

    delete[] arr; // Clean up
    return 0;
}


Overwriting red.cpp


In [74]:
!g++ -fopenmp red.cpp -o red

In [75]:
!./red

Enter the number of elements in the array: 20
Enter number of threads to use: 8

Array elements: 86 43 27 96 29 1 68 6 67 66 16 35 3 75 19 72 78 44 48 14 
Thread = 1 and i = 3
Thread = 4 and i = 12
Thread = 1 and i = 4
Thread = 4 and i = 13
Thread = 1 and i = 5
Thread = 5 and i = 14
Thread = 3 and i = 9
Thread = 3 and i = 10
Thread = 3 and i = 11
Thread = 5 and i = 15
Thread = 2 and i = 6
Thread = 6 and i = 16
Thread = 2 and i = 7
Thread = 6 and i = 17
Thread = 2 and i = 8
Thread = 7 and i = 18
Thread = 0 and i = 0
Thread = 0 and i = 1
Thread = 0 and i = 2
Thread = 7 and i = 19

Minimum value = 1
Time for finding minimum: 0.000512659 seconds
Thread = 2 and i = 6
Thread = 2 and i = 7
Thread = 2 and i = 8
Thread = 0 and i = 0
Thread = 0 and i = 1
Thread = 0 and i = 2
Thread = 3 and i = 9
Thread = 3 and i = 10
Thread = 3 and i = 11
Thread = 5 and i = 14
Thread = 5 and i = 15
Thread = 4 and i = 12
Thread = 4 and i = 13
Thread = 6 and i = 16
Thread = 6 and i = 17
Thread = 7 and i = 18
Threa